# Data Science for Social Justice Workshop: Preprocessing

* * * 

<div class="alert alert-success">  
    
### Learning Objectives 
    
* Understand why textual data needs to be preprocessed.
* Engage in common preprocessing tasks, such as lemmatization and phrase modeling.
* Distinguish between different Python packages for preprocessing.
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>

### Sections
1. [About This Workshop](#intro)
2. [Reading Data with Pandas](#read)
3. [Dropping Columns and Missing Values](#drop)
4. [Preprocessing Text Data with SpaCy](#clean)
5. [Phrase Modeling with Gensim](#gensim)
6. [Saving Data](#save)

<a id='intro'></a>

# About This Workshop

One of the most common types of data used in machine learning and data science is text data. This includes social media posts, novels, customer reviews, interview transcripts, and many others. Text is a powerful source of information, but requires specific preprocessing in order to be used in machine learning contexts. In particular, machine learning algorithms are designed to work with numerical data, so the end goal of text preprocessing is to have numeric features associated with each text in the dataset. 

In this notebook we will go over a preprocessing **pipeline** (or series of sequential steps) for text, which is the first step that we need to take in order to analyze text using our data science techniques.

This notebook is designed to help you: 

1. Read, manipulate, and write `.csv` files in a `pandas` dataframe;
2. Preprocess text with the following key skills: tokenizing, stop-word removal, n-grams extraction, and lemmatization using `spaCy`;

In Python Fundamentals, you were introduced to the basics of the `pandas` package, which we will use extensively in this notebook. This module is designed to accelerate your coding skills so that you can use Python effectively for generating research results. However, if you have less experience with Python, some of this will be overwhelming. That's totally normal, since it takes more than a couple of hours to learn how to code! 

If you are feeling overwhelmed, it can be helpful to focus on the broader purpose of the functions in the notebook for now and how we can use them to further our research purposes, rather than the details of the algorithms. It can also help to remember that you don't need to memorize every function you want to use. Even programmers who have been working with Python for years will regularly refer to documentation and resources (like this notebook!) while they are developing a project. With time and experience as you progress through the notebooks you will get more comfortable with the Python code underlying this notebook.

⚠️ **Warning:** Some of the code will take a long time to execute. The `*` on the left side of the cell will indicate that the cell is still running. The nice thing about preprocessing is that once we have the pipeline complete, we will save our results so we don't need to re-run these cells.

<a id='read'></a>

# Recap: Our Data

The first step is to read the data we are going to work with into Python so that we can work with it.

We're using the dataset taken from the subreddit [r/amitheasshole](reddit.com/r/AmItheAsshole). 

The subreddit describes itself as follows:

<img src="../../images/aita_desc.png" alt="Am I The Asshole - description" width="300"/>

The subreddit has structures in place that the community follow to come to a decision about the situation. First, OP (original poster) writes up the situation, asking AITA (Am I The Asshole). In response, for eighteen hours, the community of the subreddit will respond to the post with one of five judgments: YTA (You’re The Asshole), NTA (Not The Asshole), ESH (Everyone Sucks Here), NAH (No Assholes Here), or INFO (Not Enough Info).

💡 **Tip**: For more info on the subreddit, see [here](https://www.inverse.com/culture/am-i-the-asshole/amp). 

First, we have to read the data. We'll use a subset of the full dataset consisting of the top most popular posts, the assumption being that this will yield the most interesting results (`aita_sub_top_sm.csv`). We use `pd.read_csv()` to import the .csv file as a DataFrame.

In [28]:
# Import the pandas package
import pandas as pd 

# Read the csv file
df = pd.read_csv('../../data/aita_top_submissions.csv')

Let's have a look at look and the shape, top rows, and columns of the data.

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# This allows you to quickly see which columns you have
list(df)

This particular dataset only includes the original posts in the subreddit (so not the comments on the posts). 

There is one row per post in the dataset. The columns are as follows:

-  `idstr`: ID of the post.
- `created`: the time of the post's creation.
- `author`: Reddit author of the post.
- `title`: Title of the post.
- `selftext`: Text of the post.
- `score`: Amount of upvotes minus downvotes.
- `textlen`: Amount of words.
- `num_comments`: Amount of comments.
- `nsfw`: Flag for NSFW content.
- `flair_text`: A 'tag' that users within a subreddit can add.

🔔 **Question**: Which of these columns could contain interesting data for your research purposes?

## Removing Missing Values

First, we want to select the rows of deleted posts. On Reddit, removed posts get flagged as "[removed]" or "[deleted]". We should remove these posts, since they'll lack any text. 

We can do this using the [`isin()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html) method. We run it on the `selftext` column and include a list of phrases that indicate a removed post – that is, "[removed]" and "[deleted]".

The following line of code will select all lines that do not have 'removed' or 'deleted' in the post's text. The `~` means 'not'.

In [29]:
# Select all rows that don't have '[removed]' or '[deleted]'
df = df.loc[~df['selftext'].isin(['[removed]', '[deleted]' ]),:]
df.shape

(16325, 9)

🔔 **Question**: How many posts are left in the dataset? How many did we lose?

## Removing Null Values

Next, we need to drop **null values**. These are values that are totally missing. In this case, the web scraper may have been unable to extract text for the post. They are replaced with the value **NaN**, which stands for a null value in `pandas`. We need to deal with null values in any column that we plan to use for analysis, which in this case is the `selftext` column. We can use [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html?highlight=dropna#pandas.DataFrame.dropna) to remove the rows with null values in the target column. 

Let's use the `dropna()` method on the dataframe. We use the argument `subset`, which you set to `'selftext'`). Check the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) if you want a reminder of how this works!

In [30]:
df = df.dropna(subset='selftext')

🔔 **Question**: How many posts are left in the dataset? How many did we lose?

Let's save this cleaned-up dataframe in a new CSV.

In [ ]:
df.to_csv('../../data/aita_clean.csv')

<a id='preprocess'></a>

# Preprocessing Text Data with SpaCy

Text data collected in the real world is always going to be variable, which poses a challenge for analysis. But by reducing some of this variation, we can help improve our results. For example, if we are counting instances of the word `"weather"` in text, we might want the strings `"weather"`, `"weather."`, and `"Weather"` to all be counted as instances of the same word. However, in raw text form, these would be treated as separate strings. By performing text cleaning, we can standardize these cases and make our data easier to analyze. Some common preprocessing steps are:

- Removing punctuation
- Removing URLs
- Removing stopwords (non-content words like "a", "the", "is", etc.)
- Lowercasing
- Tokenization (e.g., splitting a sentence into distinct "chunks" or "tokens")
- Stemming, or removing the ends of words (e.g., places -> place)
- Lemmatization, or changing words to 'dictionary form' (e.g., runs, running, run -> run)

Fortunately, we don't need to code every one of these steps. Instead, we will use a package called [spaCy](https://spacy.io/) to do these things. If the text you'd like to process is general-purpose English language text (i.e., not domain-specific, like medical literature), `spaCy` is ready to use out-of-the-box. We will use the [`en_core_web_sm`](https://spacy.io/models/en/#en_core_web_sm) pipeline to cover the steps listed above. 

In [31]:
# Import spaCy
import spacy

# Load the English preprocessing pipeline
nlp = spacy.load('en_core_web_sm')

# Parse the first reddit post in the dataset
parsed_post = nlp(df.selftext[0])
print(parsed_post)

My girlfriend recently went to the beach with a few of her friends.  She has this tiny bikini bottom that is basically a thong that I HATE when she wears in public.  Well she wore it.  Not only did she wear it, she posed in the bathroom mirror of her hotel room to take a side profile picture so you could see her ass sticking out in it and posted it to her Snapchat story.   Worth mentioning I am not friends with her on Snapchat for reasons similar to this (sick of getting in fights when she says she's going out for 'girls' night then posts videos of her sitting at a table with like 5 dudes that always got invited by one of the other girls which was completely unknown to her until she arrived - most of these guys she then adds on Snapchat afterwards).  She didn't even save it and send it to me.  I saw it when she was showing me pics from her beach trip and she had screenshot that particular snap and left it in her camera roll.  Whether the ass part was intentional or not I will never kno

The base text looks the same, but we can take a closer look at `parsed_post` to see what happened under the hood. A lot of the information SpaCy has gathered operates on each **token** in the text. 

<a id='iter'></a>

# For-Loops

The strength of using computers is their speed. We can leverage this through repeated computation. In Python, we can do this using **loops**. 

A **[for loop](https://www.w3schools.com/python/python_for_loops.asp)** executes some statements once *for* each value in an iterable (like a list or a string). It says: "*for* each thing in this group, *do* these operations".

In [43]:
tokens = ['Data','science','for','social','justice']

for token in tokens:
    up = token.upper()
    print(up)

DATA
SCIENCE
FOR
SOCIAL
JUSTICE


Let's look at the syntax of this `for` loop a bit more closely:

<img src="../../images/for.svg" alt="For loop in Python" width="700"/>

Pay attention to the **loop variable** (`token`). It stands for each item in the list (`tokens`) we are iterating through. Loop variables can have any name; if we'd change it to `x`, it would still work. However, loop variables only exist inside the loop.


Let's loop over our `parsed_post` and get some **attributes** that SpaCy has extracted for us. We will save them in a list of dictionary items, then put that list into a DataFrame like we did in Python Fundamentals.

In [ ]:
# Create an empty list to store token attributes
token_data = []

# Iterate over tokens and extract attributes
for token in parsed_post:
    token_data.append({
        "text": token.text,
        "lemma": token.lemma_,
        "pos": token.pos_,
        "tag": token.tag_,
        "dep": token.dep_,
        "shape": token.shape_,
        "is_alpha": token.is_alpha,
        "is_stop": token.is_stop,
    })

# Create a pandas DataFrame from the token data
token_df = pd.DataFrame(token_data)

# Display the DataFrame
token_df

As you can see, `spaCy` does a *lot* of work. Let's have a look at the [documentation](https://spacy.io/api/attributes) to see which attributes we are looking at here.

## Preprocessing all data
Now we can scale up this process to our whole dataset. Up until this point, we have run the preprocessing pipeline on a single post, but we want to automate our code to process all posts at once. 

To do this, Let's define a few helper functions that we'll use for text normalization. In particular, the `lemmatized_sentence_corpus` generator function will use `spaCy` to:

- Iterate over the dataframe.
- Segment the threads into individual sentences.
- Remove punctuation and excess whitespace.
- Lemmatize the text.

These helper functions will automate the preprocessing for the posts in this dataset. Don't worry too much about deciphering each line of code. The main goal of these helper functions is to do a lot of the preprocessing for you so that you can use the text in analysis going forward. 

**You can find this function in the `1_Preprocessing_Project.ipynb` notebook to use on your own data.**

In [32]:
import spacy
nlp = spacy.load('en_core_web_sm')
from gensim.models.phrases import Phrases, Phraser

In [33]:
def clean(token):
    """Helper function that specifies whether a token is:
        - punctuation
        - space
        - digit
    """
    return token.is_punct or token.is_space or token.is_digit

def line_read(df, text_col='selftext'):
    """Generator function to read in text from df and get rid of line breaks."""    
    for text in df[text_col]:
        yield text.replace('\n', '')

def preprocess(df, text_col='selftext', allowed_postags=['NOUN', 'ADJ']):
    """Preprocessing function to apply to a dataframe."""
    for parsed in nlp.pipe(line_read(df, text_col), batch_size=1000, disable=["tok2vec", "ner"]):
        # Gather lowercased, lemmatized tokens
        tokens = [token.lemma_.lower() if token.lemma_ != '-PRON-'
                  else token.lower_ 
                  for token in parsed if not clean(token)]
        # Remove specific lemmatizations, and words that are not nouns or adjectives
        tokens = [lemma
                  for lemma in tokens
                  if not lemma in ["'s",  "’s", "’"] and not lemma in allowed_postags]
        # Remove stop words
        tokens = [token for token in tokens if token not in spacy.lang.en.stop_words.STOP_WORDS]
        yield tokens

Now let's run `preprocess()` over our dataframe and look at the first output. Although it looks less like coherent text, it is cleaner and will be much easier to apply natural language processing techniques.

In [34]:
# This may take a while
lemmas = [line for line in preprocess(df)]

In [ ]:
lemmas[0]

<a id='gensim'></a>

# Phrase Modeling with Gensim

Many kinds of NLP methods work better when using **N-grams**. An n-gram treats small groups of words as tokens rather than single words. This allows words that frequently appearing together to be concatenated (e.g. "new york" means something different and more specific than "new" and "york" separately). We most commonly use **bigrams** (2-word phrases) and **trigrams** (3-word phrases).

**Phrase modeling** is an approach to learning combinations of tokens that together represent meaningful multi-word concepts. So rather than treating every pair of words as a n-gram, we look for pairs that occur together frequently and identify those as n-grams. This constrains the token space by limiting the number of multi-word tokens, but requires information about what words co-occur together frequently, which is where **phrase models** come in. We can develop phrase models by looping over the the words in our lemmatized dataset and looking for words that co-occur (i.e., appear one after another) together much more frequently than you would expect them to by random chance. 

`gensim` is a popular natural language processing package. We will use it in later lessons for topic modeling and word embeddings. It also contains a [`Phrases`](https://radimrehurek.com/gensim/models/phrases.html) model that implements phrase modeling for identifying bigrams, trigrams, quadgrams, etc. `Phrases` detects phrases based on collocation counts. It builds a model of input text that you then can use on other data.

`gensim` detects a bigram if a scoring function for two words exceeds a threshold. The two important arguments to `Phrases` are `min_count` and `threshold`. The higher the values of these parameters, the harder it is for words to be combined to bigrams. We can change the value of these parameters to fine-tune our model. Try changing `min count` and `threshold` below. How does that change the output?

In [35]:
from gensim.models.phrases import Phrases, Phraser

# "Documents"
docs = ['new york is great',
        'new york is in the united states',
        'i love to stay in new york',
        'people visit the united states']
# Rudimentary tokenization
tokens = [doc.split(" ") for doc in docs]
# Create bigrams
bigram = Phrases(tokens, min_count=2, threshold=3, delimiter='_')
# Freeze bigrams and apply to data
bigram_phraser = Phraser(bigram)
[bigram_phraser[token] for token in tokens]

[['new_york', 'is', 'great'],
 ['new_york', 'is', 'in', 'the', 'united', 'states'],
 ['i', 'love', 'to', 'stay', 'in', 'new_york'],
 ['people', 'visit', 'the', 'united', 'states']]

Now let's make a bigram and trigram model for our data. Starting from the preprocessed lemmas from the `preprocess()` function above, we can use the `gensim` models to identify bigrams and trigrams in the dataset. Again, the `min_count` and `threshold` arguments can be modified to change the output of the model. 

In the code below we make a bigram model using the gensim `Phrases` object, and then build a trigram model on top of that bigram model. Finally, we use the lemmas from the preprocessed text to make a trigram model of the data. We are processing the whole dataset in this cell, so it may take a little while to run.

In [36]:
# Create bigram and trigram models
bigram = Phrases(lemmas, min_count=10, threshold=100)
trigram = Phrases(bigram[lemmas], min_count=10, threshold=50)  
bigram_phraser = Phraser(bigram)
trigram_phraser = Phraser(trigram)

# Form trigrams
trigrams = [trigram_phraser[bigram_phraser[doc]] for doc in lemmas]

In [37]:
# Join each into a string
trigrams_joined = [' '.join(trigram) for trigram in trigrams]
trigrams_joined[0]

'girlfriend recently beach friend tiny bikini basically thong hate wear public wear wear pose bathroom mirror hotel room profile picture stick post snapchat story worth mention friend snapchat reason similar sick fight girl night post video sit table like dude invite girl completely unknown arrive guy add snapchat save send pic beach trip screenshot particular snap leave camera roll ass intentional know claim like way stomach look pic beach black bikini look micro bikini bathroom mirror ocean sand friend tit ass bathroom mirror age typical year old behavior nowadays wrong think inappropriate behavior relationship asshole big_deal it?**edit guy let clear try control point time tell wear yes hate ass completely bikini tell wear problem wear general wear summer long despite groaning issue find super annoying yes issue send picture ass snapchat plain simple draw line break wear break turn sideways bathroom mirror stick butt picture send snapchat borderline nude suit wildly inappropriate cl

Once our phrase model has been trained on our total dataset, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.


In [ ]:
trigram_phraser["That", "was", "not", "a", "big", "deal"]

Let's take a look at the bigram parser. We can use `.keys()` to identify the bigrams in the dataset. How many bigrams were identified by the parser?

In [ ]:
len(bigram_phraser.phrasegrams.keys())

Let's print the first few bigrams identified in the model as well to check if they seem like appropriate bigrams. If not, we can change the parameters of the bigram model to adjust the sensitivity of the model.

In [ ]:
list(bigram_phraser.phrasegrams.keys())[:10]

In [ ]:
# Look at trigrams
[trigram for trigram in list(trigram_phraser.phrasegrams.keys()) if trigram.count('_') == 2]

<a id='save'></a>

# Saving Data

Finally, let's save our data.  In Python, **pickling** is the process of converting Python objects into binary format that can be stored or transferred. You can then reconstruct the original object from that binary format.

Pickling is useful when you want to save the state of an object so that it can be used later, or when you need to transfer an object between different Python processes.

In [38]:
import pickle

with open('../../data/aita_submission_trigrams.pickle', 'wb') as f:
    # Save (or "dump") the object into the file
    pickle.dump(trigrams_joined, f)

After using `dump()`, you'll see the data file gets added in the "data" folder, which is in the main folder of this repository. 

🔔 **Question**: Can you find the data file using a file browser (either in Jupyter or on your machine)?

⚠️ **Warning:** `aita_trigrams.pickle` is a type of file you can't just open with another program, as it's in binary format. You need to use a special pickle method called `load()` to open the pickled file and load the object:

In [ ]:
with open('../../data/aita_submission_trigrams.pickle', 'rb') as f:
    # Load the object from the file
    aita_submission_trigrams = pickle.load(f)

<div class="alert alert-success">

## ❗ Key Points

* The Pandas `DataFrame` format can be used to save Reddit data.
* Before cleaning text data, it is a good idea to drop rows and columns you don't need.
* SpaCy can be used to preprocess textual data, including tokenizatio and lemmatization.
* Gensim can be used to combine tokens into N-grams.
* Python objects can be easily saved into binary format. This is called "pickling".

</div>